In [ ]:
import keras 
import tensorflow as tf
import numpy as np

from typing import cast, Any
from keras.layers import Flatten, Conv2D, MaxPool2D, AvgPool2D, Dropout, Dense, BatchNormalization, Identity, Input, GlobalAveragePooling2D
from keras.activations import relu
from keras.optimizers import AdamW
from keras.initializers import RandomNormal

In [ ]:
#Code for Resnet block
class ResBlock(keras.layers.Layer):
    def __init__(self, num_input_layers, num_output_layers, kernel_size):
        super().__init__()
        #For first convolutional layer
        self.conv1 = Conv2D(num_output_layers, kernel_size=kernel_size, padding='same', stride=1, 
                                  kernel_initializer='glorot_uniform')
        self.bn1 = BatchNormalization()
        self.relu = relu

        #For second convolutional layer
        self.conv2 = Conv2D(num_output_layers, kernel_size=kernel_size, padding='same', stride=1, 
                                  kernel_initializer='glorot_uniform')
        self.bn2 = BatchNormalization()

        #If the input and output channels are different, we need to adjust the input
        if num_input_layers != num_output_layers:
            self.shortcut = Conv2D(num_output_layers, kernel_size=1, strides=1, padding='valid')
        else:
            self.shortcut = Identity()
    
    def call(self, x):
        #First convolutional layer
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        #Second convolutional layer
        x = self.conv2(x)
        x = self.bn2(x)

        #Add shortcut and apply non-linearity
        x += self.shortcut
        x = self.relu(x)

        return x

In [ ]:
class CNN(keras.Model):
    """Convolutional Neural Network class using Keras and Tensorflow for optimizing Hyperparameters"""
    def __init__(self, input_shape=[75, 75, 1], batch_size=64, mode='regression'):
        
        super().__init__()

        self.input_shape = tuple([batch_size] + input_shape)
        self.batch_size = batch_size
        self.input_shape = input_shape

        if (mode != 'regression') and (mode != 'classification'): #Make shure the mode is always correct
            raise ValueError("The mode of the CNN can either be 'regression' or 'classification'")
        self.mode = mode

        self.conv_layers = keras.Sequential([
                Input(shape=(self.input_shape)),
                ResBlock(num_input_layers=3, num_output_layers=16, kernel_size=3 ),
                Conv2D(16, kernel_size=2, padding='valid', strides=2),

                ResBlock(16, 32, 3),
                MaxPool2D(pool_size=(2, 2), strides=2, padding='valid'),

                ResBlock(32, 64, 3),
                MaxPool2D(pool_size=(2, 2), strides=2, padding='valid'),

                ResBlock(64, 128, 3),
                MaxPool2D(pool_size=(2, 2), strides=2, padding='valid'),

                ResBlock(128, 256, 3),
                MaxPool2D(pool_size=(2, 2), strides=2, padding='valid'),

                ResBlock(256, 512, 3),
                GlobalAveragePooling2D(data_format='channels_last')
        ])

        ############################################################################################################
        """This is the block containing Feed Forward layers for the regression part of the assignment, 
        make a similar block for classification"""

        self.regression_layers = keras.Sequential([
            Input((batch_size, 512)),
            Dense(256, activation='relu', kernel_initializer='RandomNormal'),
            Dropout(0.6),
            Dense(128, activation='relu', kernel_initializer='RandomNormal'),
            Dropout(0.4),
            Dense(64, activation='relu', kernel_initializer='RandomNormal'),
            Dropout(0.2),
            Dense(1, activation='tanh', kernel_initializer='RandomNormal')
        ])

        ###########################################################################################################
        """"You can use this to create a block that is used for classification."""
        self.classification_layers = keras.Sequential([
            Input((batch_size, 512)),
            ...
        ])
        
    def call(self, x):
        x = self.conv_layers(x)
        if self.mode == 'regression':
            x = np.abs(self.regression_layers(x))
        else:
            x = self.classification_layers(x)
        return x

